# Domestic Load Research Data Explorer for the Curious
This notebook contains the functions that do the data processing in the DLR Data Explorer app so that curious researchers can have a more fine grained view of the dataset.

## Function and Data Setup

### Import Statements

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
from dash.dependencies import Input, Output#, State

import colorlover as cl
import plotly.graph_objs as go
import plotly.offline as po
import cufflinks as cf
po.init_notebook_mode(connected=True)

import pandas as pd
import numpy as np
import os
import base64

from delprocess.surveys import loadTable, loadAnswers, searchQuestions, loadID, genS
from delprocess.loadprofiles import loadReducedProfiles
from delprocess.plotprofiles import createMap
from support import appProfiles 

mapbox_access_token = 'pk.eyJ1Ijoic2FpbnRseXZpIiwiYSI6ImNqZHZpNXkzcjFwejkyeHBkNnp3NTkzYnQifQ.Rj_C-fOaZXZTVhTlliofMA'

### Data Import
Requires datasets to be in the /data folder

In [ ]:
# Get load profile data from disk
profiles = appProfiles(1994,2014)  

# Load datasets
ids = loadID()
a_ids = ids.drop_duplicates('AnswerID')
p_ids = ids.loc[ids['Unit of measurement']==2].drop_duplicates('ProfileID')
print(ids.columns)

#a little bit of data wrangling
loc_summary = pd.pivot_table(ids, values = ['AnswerID'], index = ['Year','Survey','LocName','Lat','Long','Municipality','Province'],aggfunc = np.count_nonzero)
loc_summary.reset_index(inplace=True)
loc_summary.rename(columns={'AnswerID':'# households'}, inplace=True)

In [ ]:
tbls = loadTable('questionaires')
tbls[tbls['QuestionaireID'].isin([3, 4, 6, 7, 1000000, 1000001, 1000002])]

## Survey Locations

### Map View

In [ ]:
def create_map(ids_df):
 
    georef = pd.pivot_table(ids_df, values = ['AnswerID'], index = ['Year','LocName','Lat','Long','Municipality','Province'],aggfunc = np.count_nonzero)
    georef.reset_index(inplace=True)
    georef.rename(columns={'AnswerID':'# households'}, inplace=True)
                           
    traces = []
    for y in range(georef.Year.min(), georef.Year.max()+1):
        lat = georef.loc[(georef.Year==y), 'Lat']
        lon = georef.loc[(georef.Year==y), 'Long']
        text = georef.loc[(georef.Year==y), '# households'].astype(str) + ' household surveys</br>'+ georef.loc[(georef.Year==y), 'LocName'] + ', ' + georef.loc[(georef.Year==y), 'Municipality']
        marker_size = georef.loc[georef.Year==y,'# households']**(1/2.5)*2.7
        marker_size.replace([0,1,2,3,4, 5], 6, inplace=True)
        trace=go.Scattermapbox(
                name=y,
                lat=lat,
                lon=lon,
                mode='markers',
                marker=go.Marker(
                    size=marker_size
                ),
                text=text,
            )
        traces.append(trace)
    figure=go.Figure(
        data=go.Data(traces),
        layout = go.Layout(
                autosize=True,
                hovermode='closest',
                mapbox=dict(
                    accesstoken=mapbox_access_token,
                    bearing=0,
                    center=dict(
                        lat=-29.1,
                        lon=25
                    ),
                    pitch=0,
                    zoom=4.32,
                    style='light'
                ),
                margin = go.Margin(
                        l = 10,
                        r = 10,
                        t = 20,
                        b = 30
                ),
                showlegend=True
            )
    )
    return po.iplot( figure)

In [ ]:
create_map(ids)

### Total surveyed households by year

In [ ]:
def plotCustomerDist(ids_df, id_filter, **kwargs):
    ids = ids_df.groupby(['Survey','Year'])[id_filter].nunique()
    
    if 'nrslr_col' in kwargs:
        nrslr_col = kwargs['nrslr_col']
    else: nrslr_col = 'red'
    if 'eskomlr_col' in kwargs:
        eskomlr_col = kwargs['eskomlr_col'] 
    else: eskomlr_col = 'blue' 
    
    nrslr = go.Bar(x = ids['NRS LR'].index, 
                   y = ids['NRS LR'].values,
                   marker=dict(color=nrslr_col),
                   name = 'Municipalities')

    eskomlr = go.Bar(x = ids['Eskom LR'].index, 
                     y = ids['Eskom LR'].values,
                     marker = dict(color=eskomlr_col),
                     name = 'Eskom')

    layout = go.Layout(title=kwargs['plot_title']+' from 1994 - 2014',
                       barmode = 'relative',
                       xaxis=dict(title='Year', tickvals=list(range(1994,2015))),
                       yaxis=dict(title=id_filter+' Count', showline=True),
                       margin=dict(t=70),
                       height=450, width=850)

    fig = go.Figure(data=[nrslr, eskomlr], layout=layout)
    po.iplot(fig)

In [ ]:
plotCustomerDist(p_ids, 'ProfileID', plot_title = 'Households Metered')

In [ ]:
plotCustomerDist(a_ids, 'AnswerID', plot_title = 'Household Surveys Conducted')

In [ ]:
plotCustomerDist(p_ids, 'Location', plot_title='Sites Metered', nrslr_col='tomato', eskomlr_col='skyblue')

In [ ]:
plotCustomerDist(a_ids, 'Location', plot_title='Sites Surveyed', nrslr_col='tomato', eskomlr_col='skyblue')

In [ ]:
ids.groupby(['Survey','Province','Year'])['AnswerID'].count().unstack().fillna('')

### Total surveyed household count by municipality

In [ ]:
pd.DataFrame(loc_summary.groupby(['Province','Municipality'])['# households'].sum())

### Data Collection by Municipal Collaborators
*DCOs: Data Collection Organisations.These have been captured under NRS LR Survey in the database

In [ ]:
ids.loc[ids.Survey=='NRS LR'].groupby('Municipality')[['Location','AnswerID']].agg({'Location':'nunique','AnswerID':'nunique'})#.unstack().fillna('')

## Socio-demographic Summaries

### Search Survey Questions for Key Words

In [ ]:
searchQuestions('hotwater')

### Create Dataframe with Selected Features

In [ ]:
sd = genS('dist_base_appliance_behaviour',1994,2014)
sd.head()

### Distribution of Survey Variables for Households

In [ ]:
def plotVarDist(var_df, var, var_bin, var_col, **kwargs):
    v = [go.Histogram(x=var_df[var], xbins=dict(
                                    start=0,
                                    end=var_df[var].max()/2, #exclude the long tail from analysis
                                    size= var_bin),
                                    autobinx=False,
                                    marker=dict(color=var_col, line=dict(color='#000000', width=0.5)))]
    layout = go.Layout(title=kwargs['plot_title'],
                      xaxis=dict(title=kwargs['x_title']),
                      yaxis=dict(title='Household Count', showline=True),
                      margin=dict(t=70),
                      height=350, width=650)
    fig = go.Figure(data=v, layout=layout)
    po.iplot(fig)

In [ ]:
plotVarDist(sd, 'monthly_income', 2000, cl.scales['3']['qual']['Pastel1'][0], 
            plot_title='Distribution of Monthly Income of Survey Respondents', 
            x_title='Household Income in ZAR (R2k bin size, inflation adjusted to Dec 2016)')

In [ ]:
plotVarDist(sd, 'floor_area', 25, cl.scales['3']['qual']['Pastel1'][1], 
            plot_title='Distribution of Dwelling Floor Area of Survey Respondents', 
            x_title='Dwelling Floor Area (25mˆ2 bin size)')

In [ ]:
plotVarDist(sd, 'years_electrified', 1, cl.scales['3']['qual']['Pastel1'][2], 
            plot_title="Survey Respondents' Years Since Electrification", 
            x_title="Years Electrified (1 year bin size)")

#### View Locations for Year

In [ ]:
#Specify Survey Year
year = 2005

ids_yr = ids[ids.Year==year]
list(ids_yr.LocName.unique())

### Location Details

In [ ]:
location = 'Dinokana'

aid_select = ids_yr.loc[(ids_yr.LocName==location)&(ids_yr.AnswerID!=0), 'AnswerID']
sd_yr = sd[sd.AnswerID.isin(aid_select)]
sd_yr.describe()

#### Electricity Supply

In [ ]:
sd_yr[['years_electrified','main_switch']].describe()

#### Appliance Ownership

In [ ]:
pd.DataFrame(sd_yr.iloc[:,4:4+len(appliances)].count()/len(sd_yr), columns=['Ownership (%hh)'])

#### Size of Housing

In [ ]:
sd_yr[['floor_area','rooms']].describe()

In [ ]:
mdf = ids[ids.AnswerID!=0].merge(sd)
Eskom_income = mdf[mdf.Survey=='Eskom LR'].groupby(['Year','Province']).aggregate({'AnswerID':np.count_nonzero,'monthly_income':np.mean})

In [ ]:
mdf.loc[mdf.Survey=='Eskom LR',['Year','monthly_income']].pivot(columns='Year').iplot(kind='box', title='Monthly income distribution for Eskom Sites')

In [ ]:
mdf.loc[mdf.Survey=='NRS LR',['Year','monthly_income']].pivot(columns='Year').iplot(kind='box', title='Monthly income distribution for Municipal Sites')

In [ ]:
df = mdf.loc[mdf.Survey=='Eskom LR',['Year','monthly_income']].pivot(columns='Year')

In [ ]:
df.columns

In [ ]:
df.columns.names

In [ ]:
df.reset_index(drop=True, col_level='Year')